# TwitterScraper

- chrome driver: https://chromedriver.chromium.org/downloads

In [1]:
#requests
import requests
import urllib
import urllib.request
from urllib.request import urlopen
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
from datetime import datetime
import requests
from urllib.request import urlopen, Request

#data, strucuture and maths
import pandas as pd
import numpy as np
import math
import json
import string
from  more_itertools import unique_everseen
import random

#progress,performance and management
from tqdm.notebook import tqdm
import threading
import os
import ssl
from IPython.display import clear_output
import platform
import os

# imports used in Selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Chrome, ChromeOptions

#time
from time import sleep
import time

#text processing / regex
import regex
import re

#make wide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#passwords
import getpass

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 5000)
pd.options.display.float_format = '{:.3f}'.format

In [5]:
class TwitterScraper():

    def __init__(self, driver_loc='/Users/takapy/Chromedriver/chromedriver'):
        self.driver_loc = driver_loc
        self.userDetails
        self.openWebdriver
        self.closeWebdriver
        self.twitterLogin
        self.logIn
        self.setTarget
        self.scrapeLinks
        self.getLinks
        self.multithreadCompile
        self.multithreadExecute
        self.getData
        self.postUser
        self.postComment
        self.postUrl
        self.getDatahtml
        self.getHashtags


    def twitterLogin(self, driver):    
        url = "https://twitter.com/login/error?username_or_email=%40"
        driver.get(url+self._username)
        time.sleep(2)

        element=driver.find_elements_by_xpath('//input[@autocapitalize="none"]')[1]
        element.send_keys(self._password)
        element.send_keys(Keys.ENTER)

        return driver

    
    #extracts hashtags from any string returning list of hashtags
    def getHashtags(self, _string):

        #splits string into list and appends unique hashtags into a new list
        hashtags = [hashtag for hashtag in set([token for token in str(_string).split() if token.startswith("#")])]

        #if there are hashtags in the string we process them further..
        if len(hashtags) > 0:
            #this will break up any hashtags that haven't been seperated by a space
            hashtags_seperated = [i for i in ''.join(hashtags).strip().split('#') if len(i) > 0]
            #this will remove any punctuation
            hashtags_clean = [hashtag.translate(str.maketrans('', '', string.punctuation)) for hashtag in hashtags_seperated]
            hashtags_clean = [i.lower() for i in hashtags_clean]
            #returns unique, cleaned hashtags without the
            return ' '.join(list(set(hashtags_clean)))

        else:
            return np.nan

        
    def userDetails(self):
        username = input('Enter username...')
        password = getpass.getpass('Enter password...')
        self._password= password
        self._username = username
        return
    
    
    def openWebdriver(self):
        # intiate driver
        print("Launching driver...")
        driver = webdriver.Chrome(self.driver_loc)
        return driver
    
    
    def closeWebdriver(self,driver):
        driver.close()
        return

    
    # get user details, log in and initiate driver
    def logIn(self):
        self.userDetails()
        driver = self.openWebdriver()
#         url = 'https://twitter.com/'
#         driver.get(url)
#         self.activedriver = driver
        self.activedriver = self.twitterLogin(driver)
        clear_output()
        print('Successfully logged in..ready to scrape')
        
        
    """
    Functions that set either a profile or a hashtag as a target and then
    scrapes user specified number of post links
    """
    def setTarget(self):
        hashtag = input('Which hashtag do you want to scrape posts for: ')
        self.target_label = '#' + hashtag
        self._target = 'https://twitter.com/search?q=%23' + hashtag + '%20lang%3Aja&f=live'
#         self._target = 'https://twitter.com/search?q=%23' + hashtag + '%20lang%3Aja'
        return self._target
    

    def scrapeLinks(self, url):
        # pass url as argument to Selenium webDriver, loads url
        self.activedriver.get(url)
        options = webdriver.ChromeOptions()
        
        # start maximised
        options.add_argument("--start-maximized")

        # gets scroll height
        last_height = self.activedriver.execute_script("return document.body.scrollHeight")

        # initiate empty list for unique Instagram links
        links = []

        #some lines for user interactivity / selection of link target(n)
        target = input("How many links do you want to scrape (minimum)?: ")
        print('terget size:{}'.format(target))
        print("Staring Selenium scrape, please keep browser open.")

        #this loops round until n links achieved or page has ended

        while True:
            source = self.activedriver.page_source
            data= BeautifulSoup(source, 'html.parser')
            body = data.find('body')
            for link in body.findAll('a'):
                if re.match("/", link.get('href')):
                    if 'status' in link.get('href'):
                        if 'photo' not in link.get('href'):
                            links.append('https://twitter.com/' + link.get('href'))
                else:
                    continue

            # Scroll down to bottom
            self.activedriver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load page
            time.sleep(3)
            # Calculate new scroll height and compare with last scroll height
            new_height = self.activedriver.execute_script("return document.body.scrollHeight")

            # if no more content, scrape loop is terminated
            if new_height == last_height:
                print('no more content')
                break

            last_height = new_height

            # update on successful links scraped
            print("Scraped ", len(links)," links, ", len(set(links)),' are unique')

            # if n target met then while loop breaks
            if len(set(links))>int(target):
                break

        #links are saved as an attribute for the class instance
        self._links = list(unique_everseen(links))

        #clear the screen and provide user feedback on performance
#         clear_output()

        print("Finished scraping links. Maxed out at ", len(links)," links, of which ", len(self._links),' are unique.')
        print("Unique links obtained. Closing driver")

        # close driver
        self.closeWebdriver(self.activedriver)

        return
    
    
    # get all the unique links
    def getLinks(self):
        return self.scrapeLinks(self.setTarget())


    """
    Multi threading functions
    """
    def multithreadCompile(self,thread_count,iteration_list,func):
        jobs = []
        #distribute iteration list to batches and append to jobs list
        batches = [i.tolist() for i in np.array_split(iteration_list,thread_count)]
        for i in range(len(batches)):
            jobs.append(threading.Thread(target=func,args=[batches[i]]))
        return jobs

    def multithreadExecute(self,jobs):
        # Start the threads
        for j in jobs:
            print('execute working')
            j.start()

        # Ensure all of the threads have finished
        for j in jobs:
            j.join()
        return


    # parser html in any given unique Instagram URL
    def getDatahtml(self,url):
        page = urlopen(url).read()
        data = BeautifulSoup(page, 'html.parser')
        return data
    
    
    """
    Methods that extract various fields of data from Instagram JSON dictionaries
    """
    # get user name
    def postUser(self, data):
        return data.find('meta', attrs={'property': 'og:title'}).get('content').replace(' on Twitter', '')

    # get post comment
    def postComment(self, data):
        return data.find('meta', attrs={'property': 'og:description'}).get('content').replace('“', '').replace('”','')

    #return original post link
    def postUrl(self, data):
        return data.find('meta', attrs={'property': 'og:url'}).get('content')
    
    #return unix time
    def postDatetime(self, data):
        date_time = ''
        for link in data.findAll('span', class_="_timestamp js-short-timestamp js-relative-timestamp"):
            date_time = link.get('data-time')
        return date_time
    
    
    # extract data and return dataframe
    def getData(self):
        post_user_l = []
        post_l = []
        post_url_l = []
        post_time_l = []
        
        self._listStack = [post_user_l, post_url_l, post_l, post_time_l]
        
        self._functionStack = [self.postUser, self.postUrl, self.postComment, self.postDatetime]

        def extractData(links=self._links):

            for i in tqdm(range(len(links))):
                try:
                    data = self.getDatahtml(links[i])
                    for function in self._functionStack:
                        try:
                            self._listStack[self._functionStack.index(function)].append(function(data))
                        except:
                             self._listStack[self._functionStack.index(function)].append(np.nan)
                except:
                    pass
            return

        # execute html parsing fuction using multi threading
        print("Attemping multi-threading...")
#         threads = int(input("How many threads?: "))
        threads = 6
        print("Executing...")

        self.multithreadExecute(self.multithreadCompile(threads,self._links,extractData))

        #set up intial data structure
        print('post_url_l length: {}'.format(len(post_url_l)))
        print('post_l length: {}'.format(len(post_l)))
        print('post_user_l length: {}'.format(len(post_user_l)))
        print('post_time_l length: {}'.format(len(post_time_l)))

        df = pd.DataFrame({'searched_for':[self.target_label]*len(post_l),
                           'post_url' :post_url_l,
                           'post':post_l,
                           'user':post_user_l,
                           'post_time':post_time_l,
                            })

        df['hashtags'] = df['post'].map(self.getHashtags)
        df.reset_index(drop=True,inplace=True)

        self._df = df
        return df

In [6]:
scraper = TwitterScraper()

In [8]:
scraper.logIn()

Successfully logged in..ready to scrape


link: https://twitter.com/search?q=%23python%20lang%3Aja

In [9]:
scraper.getLinks()

Which hashtag do you want to scrape posts for:  python
How many links do you want to scrape (minimum)?:  3000


terget size:3000
Staring Selenium scrape, please keep browser open.
Scraped  8  links,  8  are unique
Scraped  31  links,  31  are unique
Scraped  49  links,  44  are unique
Scraped  69  links,  62  are unique
Scraped  89  links,  71  are unique
Scraped  110  links,  86  are unique
Scraped  126  links,  102  are unique
Scraped  142  links,  114  are unique
Scraped  157  links,  129  are unique
Scraped  174  links,  146  are unique
Scraped  190  links,  158  are unique
Scraped  205  links,  173  are unique
Scraped  225  links,  190  are unique
Scraped  243  links,  208  are unique
Scraped  263  links,  220  are unique
Scraped  287  links,  235  are unique
Scraped  311  links,  244  are unique
Scraped  330  links,  256  are unique
Scraped  349  links,  273  are unique
Scraped  367  links,  287  are unique
Scraped  387  links,  296  are unique
Scraped  407  links,  310  are unique
Scraped  429  links,  328  are unique
Scraped  449  links,  338  are unique
Scraped  466  links,  352  are un

In [10]:
df = scraper.getData()

Attemping multi-threading...
Executing...
execute working
execute working
execute working
execute working
execute working
execute working








post_url_l length: 1196
post_l length: 1196
post_user_l length: 1196
post_time_l length: 1196


In [11]:
display(df.head(), df.shape)

,searched_for,post_url,post,user,post_time,hashtags
0,#python,nan,nan,nan,,nan
1,#python,nan,nan,nan,,nan
2,#python,nan,nan,nan,,nan
3,#python,nan,nan,nan,,nan
4,#python,nan,nan,nan,,nan


(1196, 6)

In [12]:
# 欠損データは削除
df.dropna(inplace=True) #行方向に削除

In [13]:
# hasgtagが1つしかついていないデータも削除
df['hashtag_len'] = df['hashtags'].map(lambda x: len(x.split()))
df = df.query('hashtag_len >= 2')

In [14]:
# 特定の単語を含むデータも削除
# word = 'python'
# df = df[~df['hashtags'].str.contains(word)]

# word = '人工知能'
# df = df[~df['hashtags'].str.contains(word)]

# word = 'データサイエンティスト'
# df = df[~df['hashtags'].str.contains(word)]

In [15]:
display(df.head(), df.shape)

,searched_for,post_url,post,user,post_time,hashtags,hashtag_len
0,#データサイエンティスト,https://twitter.com/K03475916/status/1257810582448689154,#今日の積み上げ\n・Udemy中級編7.8\n・DS養成講座30回31回課題\n・progate python3章\n・筋トレ(有酸素運動)\n\n本日もコツコツ積み上げ😊18時に全て終わらせる！と時間を設定したほうがダラダラしなくて済みますよね👌\n\n#駆け出しエンジニアと繋がりたい\n#データサイエンティスト\n#筋トレ,K,1588720386,筋トレ 今日の積み上げ 駆け出しエンジニアと繋がりたい データサイエンティスト,4
2,#データサイエンティスト,https://twitter.com/ponpoko_code/status/844690156091265024,データ分析って給料高いな～。人材不足らしい。統計学、プログラミングに加えて人工知能も出来ると完璧だ。アカデミックな内容も強いし習得するまで根気がいるな～。\n\n#データサイエンティスト #人工知能 #統計学 #プログラミング #ダイヤモンド,ぽんぽこ@プロぽこ-プログラミング,,統計学 人工知能 ダイヤモンド データサイエンティスト プログラミング,5
3,#データサイエンティスト,https://twitter.com/sazan_dora/status/797434083248721921,データサイエンティストになりたい人向けまとめ - NAVER まとめ https://t.co/yW6Tg4EZzC\n\n#データマイニング #データサイエンス #データサイエンティスト,サザンドラ,,データマイニング データサイエンス データサイエンティスト,3
4,#データサイエンティスト,https://twitter.com/WXBC_jp/status/1130260436291424256,【#野球 ネタ⚾️】\n\n#ヤクルトスワローズ の #山田哲人 選手の成績データを、気温・湿度の #気象 データを照らし合わせ、「蒸暑い日の方がよく打つ」と導いた自由研究✍️\n\n将来は #データサイエンティスト に⁉️\n今後も、気象データを存分に利活用してくださいね😍\n※夏男って昔からいて面白いですよね… https://t.co/97JkdiM0nS,"気象ビジネス推進コンソーシアム（略称：WXBC,事務局：気象庁）",,山田哲人 気象 データサイエンティスト ヤクルトスワローズ,4
6,#データサイエンティスト,https://twitter.com/hamahiroshi164/status/1257679355163013120,「データサイエンティストを目指す人のzoom飲み会」終わりました💡\n\n現場のリアルが知れる、大変有意義な会でした😌\n\nみなさんご丁寧に教えてくださってありがとうございました🙇‍♀️\n\n今すごく穏やかな気持ちです\n\n#データサイエンティスト\n#プログラミング初心者,DSになる男ハマヒロシ30歳子煩悩,1588689099,プログラミング初心者 データサイエンティスト,2


(1489, 7)

In [16]:
# 保存
cols = ['searched_for', 'post', 'hashtags', 'hashtag_len']
df[cols].to_csv('twitter_python.csv', index=False)

```html
<link href="https://puchiaqua/status/1252558925871722497" media="handheld, only screen and (max-width: 640px)" rel="alternate"/>
<link href="android-app://com.twitter.android/twitter/tweet?status_id=1252558925871722497&amp;ref_src=twsrc%5Egoogle%7Ctwcamp%5Eandroidseo%7Ctwgr%5Estatus%7Ctwterm%5E1252558925871722497" rel="alternate"/>
<link href="/opensearch.xml" rel="search" title="Twitter" type="application/opensearchdescription+xml"/>
<link id="async-css-placeholder"/>
<meta content="twitter://status?id=1252558925871722497" property="al:ios:url"/>
<meta content="333903271" property="al:ios:app_store_id"/>
<meta content="Twitter" property="al:ios:app_name"/>
<meta content="twitter://status?status_id=1252558925871722497" property="al:android:url"/>
<meta content="com.twitter.android" property="al:android:package"/>
<meta content="Twitter" property="al:android:app_name"/>
<meta content="article" property="og:type"/>
<meta content="https://twitter.com/puchiaqua/status/1252558925871722497" property="og:url"/>
<meta content="みつきタン on Twitter" property="og:title"/>
<meta content="https://pbs.twimg.com/profile_images/1134906748559474688/Jw9zZ6Ns_400x400.png" property="og:image"/>
<meta content="“【Windows】さくらのVPS for Windowsを借りてみた【家パソよりお得！？】 https://t.co/ZqdppwpcZO
#仮想環境 #さくらサーバー #クラウド #vps #windows #プログラミング勉強環境 #初心者向け #Python”" property="og:description"/>
<meta content="Twitter" property="og:site_name"/>
<meta content="2231777543" property="fb:app_id"/>
```

In [15]:
page = urlopen('https://twitter.com/bou999999999/status/1292389855222272006').read()
data = BeautifulSoup(page, 'html.parser')

In [16]:
data

<!DOCTYPE html>

<html dir="ltr" lang="en">
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
<link href="//abs.twimg.com" rel="preconnect"/>
<link href="//api.twitter.com" rel="preconnect"/>
<link href="//pbs.twimg.com" rel="preconnect"/>
<link href="//t.co" rel="preconnect"/>
<link href="//video.twimg.com" rel="preconnect"/>
<link href="//abs.twimg.com" rel="dns-prefetch"/>
<link href="//api.twitter.com" rel="dns-prefetch"/>
<link href="//pbs.twimg.com" rel="dns-prefetch"/>
<link href="//t.co" rel="dns-prefetch"/>
<link href="//video.twimg.com" rel="dns-prefetch"/>
<link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/client-web-legacy/polyfills.b0114665.js" nonce="ZTA5NmE3NzgtYjU2ZS00MDY4LWExNjMtOTEzZWU2N2Y4ZmVl" rel="preload"/>
<link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/client-web-legacy/vendors~main.5f094565.js" nonc

In [14]:
date_time = ''
for link in data.findAll('span', class_="_timestamp js-short-timestamp js-relative-timestamp"):
    date_time = link.get('data-time')

date_time

''